In [ ]:
import requests
import pandas as pd
import json
import html2markdown
import nbformat as nbf
import os
import getpass
import ipywidgets as widgets
from time import sleep

## List of Problems

We retreive the list of problems on LeetCode and display them in a dataframe.

In [ ]:
problems = requests.get('https://leetcode.com/api/problems/all/').json()['stat_status_pairs']

problem_list = []
for i in problems:
    problem_list.append([i['stat']['frontend_question_id'], i['stat']['question__title'], i['stat']['question__title_slug']])
    
LeetCode = pd.DataFrame(problem_list, columns = ['#' , 'Problem', 'titleSlug'])
LeetCode = LeetCode.sort_values(by=['#']).reset_index(drop=True)

LeetCode.head()

## Querying problem description

We first submit a query to retrieve the problem description (_does not work for problems exclusive to premium members_).

In [ ]:
def problem_description(titleSlug, headers):
    
    # Update header
    headers['Referer'] = 'https://leetcode.com/problems/' + titleSlug + '/'
    
    # Graphql query url
    query_url = 'https://leetcode.com/graphql'

    # Define request payload
    payload = {'query': 
    '''{
      question(titleSlug: "''' + titleSlug + '''\") 
      {
        titleSlug
        questionId
        title
        content
        codeSnippets
        {
          langSlug
          code
        }
        difficulty
        topicTags 
        {
          slug
        }
      }
    }'''
    }
    
    # Make the query
    response = requests.post(query_url, params=payload, headers=headers).json()['data']['question']
    
    return response

## Generating Notebooks

The following code automates the generation of a jupyter notebook given the query response.

In [ ]:
def generate_notebook(response, lang='python3', code=None, submission=None):
    
    fname = '{:04d}. {:s}'.format(int(response['questionId']), response['title'])
    
    url = 'https://leetcode.com/problems/' + response['titleSlug'] + '/'
                                  
    difficulty = response['difficulty']
    
    tags = []
    for i in response['topicTags']:
        tags.append('[' + i['slug'] + ']' + '(https://leetcode.com/problemset/all/?topicSlugs=' + i['slug'] + ')')
                                  
    content = response['content']
    
    code_dict = {'cpp': 0, 'java': 1, 'python3': 3}
    if code==None:
        try:
            code = response['codeSnippets'][code_dict[lang]]['code']
        except:
            code = ""
    
    variables = {
        'fname': fname,
        'url': url,
        'difficulty': difficulty,
        'tags': tags
    }
    
    description = '''## Problem [{fname}]({url})\n\n__Difficulty__: {difficulty}\n\n__Tags__: {tags}\n\n'''.format(**variables) + html2markdown.convert(content)
    
    if submission==None:
        submission = '## Solution\n\n'
    else:
        submission = '## [Solution](' + submission + ')\n\n'
    
    nb = nbf.v4.new_notebook()
    nb['cells'] = [nbf.v4.new_markdown_cell(description),
                   nbf.v4.new_markdown_cell(submission),
                   nbf.v4.new_code_cell(code)]
    
    # Kernel specifications for the generated notebook
    if lang == 'cpp':
        kernelspec = {'display_name': 'C++17', 'language': 'C++17', 'name': 'xeus-cling-cpp17'}
    elif lang == 'java':
        kernelspec = {'display_name': 'SciJava', 'language': 'groovy', 'name': 'scijava'}
    else:
        kernelspec = {'display_name': 'Python 3', 'language': 'python', 'name': 'python3'}
    
    nb['metadata'] = {'kernelspec': kernelspec}
    
    with open('Problems/' + fname + '.ipynb', 'w') as f:
        nbf.write(nb, f)

## Executing

Here we define the headers for the query request and iterate through the entire list of problems to generate notebooks complete with problem description and code snippets.

The following coding languages are supported.

- `cpp`
- `java`
- `python3`


In [ ]:
# Define headers
headers = {}
response = requests.get('https://leetcode.com/accounts/login/')

__cfduid = response.cookies['__cfduid']
csrftoken = response.cookies['csrftoken']

headers['Cookie'] = '__cfduid=' + __cfduid + '; ' + 'csrftoken=' + csrftoken
headers['x-csrftoken'] = csrftoken

# Coding lang: can be either 'cpp', 'java', 'python3'
code_display = {'cpp': 'C++', 'java': 'Java', 'python3': 'Python 3'}
inv_code_display = {v: k for k, v in code_display.items()}

class code_update:
    def __init__(self):
        self.widget_value = 'C++'
        self.value = inv_code_display[self.widget_value]

    def update(self, val_dict) -> None:
        self.widget_value = val_dict['new']
        self.value = inv_code_display[self.widget_value]

lang_selector = code_update()

lang_buttons = widgets.ToggleButtons(
    options = ['C++', 'Java', 'Python 3'],
    tooltips =['cpp', 'java', 'python3'],
    value = 'Python 3',
    description = 'Language:',
    disabled = False,
)

def on_change(val_dict):
    lang_selector.update(val_dict)
    
lang_buttons.observe(on_change, names='value')

display(lang_buttons)

In [ ]:
try:
    os.makedirs('Problems')
except:
    print('Problems directory already exists.')

exceptions = []

for i in range(len(LeetCode)):
    titleSlug = LeetCode.iloc[i][2]
    
    fname = '{:04d}. {:s}'.format(LeetCode.iloc[i][0], str(LeetCode.iloc[i][1]))
    
    if os.path.exists('Problems/' + fname + '.ipynb'):
        continue
    
    description = problem_description(titleSlug, headers)
    
    if description['content'] is not None:
        generate_notebook(description, lang_selector.value)
    else:
        exceptions.append(fname)
        
with open('Premium Problems.txt', 'w') as f:
    for i in exceptions:
        f.write('{:s}\n'.format(i))

## Login to retrieve submissions

In order to retrieve submissions, we need to create a login session. The login function prompts for your LeetCode username and password.

In [ ]:
def login(username, password):
    url = 'https://leetcode.com/accounts/login/'

    headers = {
        'Accept': '*/*',
        'Connection': 'keep-alive',
        'Host': 'leetcode.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
        'Referer': 'https://leetcode.com/accounts/login/',
    }

    response = requests.get(url)
    __cfduid = response.cookies['__cfduid']
    csrftoken = response.cookies['csrftoken']

    headers['Cookie'] = '__cfduid=' + __cfduid + '; ' + 'csrftoken=' + csrftoken
    headers['x-csrftoken'] = csrftoken

    session = requests.session()

    login_data={}

    login_data['csrfmiddlewaretoken'] = csrftoken
    login_data['login'] = username
    login_data['password'] = password

    res = session.post(url, headers=headers, data=login_data)
    if res.text.find(username)>0:
        print ('Login successful.')
        return session
    else:
        print ('Login unsuccessful.')


username = input ('LeetCode username: ')
password = getpass.getpass(prompt='LeetCode password: ')

session = login(username, password)

## List of Problems

We will retreive the list of problems again, now with submission status.

In [ ]:
problems = session.get('https://leetcode.com/api/problems/all/').json()['stat_status_pairs']

difficulty_dict = {1: 'Easy', 2: 'Medium', 3: 'Hard'}

problem_list = []
for i in problems:
    problem_list.append([i['stat']['frontend_question_id'], i['stat']['question__title'], i['stat']['question__title_slug'], difficulty_dict[i['difficulty']['level']],  i['status']])
    
LeetCode = pd.DataFrame(problem_list, columns = ['#' , 'Problem', 'titleSlug', 'Difficulty', 'Status'])
LeetCode = LeetCode.sort_values(by=['#','Status']).reset_index(drop=True)

LeetCode.head()

## Query Submission

In a login session, we can query the submission stats.

In [ ]:
def retrieve_submission(titleSlug):
    query_url = 'https://leetcode.com/api/submissions/{:s}/?offset={:d}&limit={:d}&lastkey='.format(titleSlug, 0, 0)
    n = 0
    while n<10:
        n += 1
        try:
            response = session.get(query_url).json()['submissions_dump']
            return response
        except:
           sleep(0.05)

## Generate entry for README.MD

Generate a table entry for the submission stats.

In [ ]:
code_display = {'cpp': 'C++', 'java': 'Java', 'python3': 'Python 3'}

def generate_entry(description, submission):
    questionId = description['questionId']
    title = '[{:s}](Problems/{:04d}.%20{:s}.ipynb)'.format(description['title'], int(questionId), description['title'].replace(' ', '%20'))
    lang = code_display[submission.loc[0]['lang']]
    solution = '[{:s}](https://leetcode.com{:s})'.format(lang, submission.iloc[0]['url'])
    difficulty = description['difficulty']
    tags = ''
    for i in description['topicTags']:
        tags += ('[' + i['slug'] + ']' + '(https://leetcode.com/problemset/all/?topicSlugs=' + i['slug'] + ')') + ', '
    tags = tags[:-2]
    stats = '__{:s}__, {:s}'.format(submission.iloc[0]['runtime'], submission.iloc[0]['memory'])

    entry = '| {:04d} | {:s} | {:s} | {:s} | {:s} | {:s} |'.format(int(questionId), title, solution, difficulty, tags, stats)
    
    return entry

def generate_readme(submission_stats):
    count = 0
    easy_count = 0
    medium_count = 0
    hard_count = 0
    
    for i in submission_stats:
        count += 1
        if i.find('Easy')>0:
            easy_count += 1
        if i.find('Medium')>0:
            medium_count += 1
        if i.find('Hard')>0:
            hard_count += 1
                
    total = len(LeetCode)
    
    n_easy = len(LeetCode.loc[LeetCode['Difficulty'] == 'Easy'])
    n_medium = len(LeetCode.loc[LeetCode['Difficulty'] == 'Medium'])
    n_hard = len(LeetCode.loc[LeetCode['Difficulty'] == 'Hard'])
    
    readme = '''<p align='center'><img width='320' src='https://theme.zdassets.com/theme_assets/9008406/036323c6afd10392aa5b7e3a2eb7557d17955c81.png'></p>'''
    readme += '\n\n'
    readme += '### Current Progress:\n'
    readme += '<img src=\'https://img.shields.io/static/v1.svg?label=Total&message={:d}/{:d}&color=black&style=square&logo=leetcode\'>'.format(count, total)
    readme += ' <img src=\'https://img.shields.io/badge/Easy-{:d}%2F{:d}-red.svg\'>'.format(easy_count, n_easy)
    readme += ' <img src=\'https://img.shields.io/badge/Medium-{:d}%2F{:d}-green.svg\'>'.format(medium_count, n_medium)
    readme += ' <img src=\'https://img.shields.io/badge/Hard-{:d}%2F{:d}-blue.svg\'>'.format(hard_count, n_hard)
    readme += '\n\n'
    readme += '| # | Title | Solution | Difficulty | Tags | Runtime/Memory |\n'
    readme += '|:-:|:-----:|:--------:|:----------:|:----:|:--------------:|\n'
    for i in submission_stats:
        readme += i + '\n'

    return readme

## Complete Notebooks with Submitted Code/ Generate README.MD

For accepted questions, we retrieve our submitted code and add them to the notebooks.

Submission records are also added to the [README.MD](./README.MD).

In [ ]:
submission_stats = []

for i in range(len(LeetCode)):
    if LeetCode.iloc[i][4] is not None:
        titleSlug = LeetCode.iloc[i][2]
        submission = retrieve_submission(titleSlug)
        if submission is None:
            continue
        
        submission = pd.DataFrame.from_dict(submission).drop(columns=['compare_result', 'is_pending', 'time', 'timestamp'])
        submission = submission.sort_values(['runtime','memory'], ascending=[1,1]).reset_index(drop=True)
        
        submission_url = 'https://leetcode.com' + submission.iloc[0]['url']
        
        description = problem_description(titleSlug, headers)
        
        # generate_notebook(description, lang=lang, code=code, submission=submission_url)
        
        submission_stats.append(generate_entry(description, submission))

readme = generate_readme(submission_stats)

with open('README.md', 'w') as f:
    f.write(readme)